In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from src.data_processing import load_data
import itertools
import string
from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag
from src.data_processing import load_cmu
from nltk.stem import PorterStemmer

In [17]:
import codecs
import json
import itertools
import numpy
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from numpy.ma import dot
from collections import defaultdict
import pygtrie as trie
from src.pronunciations import phonetic_translation

In [4]:
cmu = load_cmu()
data = load_data()

In [5]:
from src.ngrams import ngram_frequency
ngram_frequency("clearing out the cache".split())

202

In [64]:
task2 = data[1]
print('\n'.join([' '.join(p['words']) for p in task2 if 'Tom' in p['words']]))

' ' I ' m halfway up a mountain , ' ' Tom alleged .
I ' d like to be a Chinese laborer , said Tom coolly .
It ' s between my sole and my heel , said Tom archly .
I deny everything , said Tom knowingly .
' ' I ' ve stuck a pin through my nose , ' ' said Tom punctually .
Let ' s all play an A , a C sharp , and an E , cried Tom ' s band with one accord .
The doctors had to remove a bone from my arm , said Tom humorlessly .
I ' ve been to a film festival in the South of France , said Tom cannily .
' ' Why is this telephone flex always tangled ? ' ' asked Tom coyly .
' ' I ' ve been listening to the Brandenburg Concertos , ' ' Tom barked .
' ' I can see through the window , ' ' said Tom stiltedly .
' ' This food tastes of plutonium , ' ' said Tom glowingly .
' ' Would you like some soda ? ' ' asked Tom caustically .
I ' m a lion hunter , said Tom , pridefully
I ' m from a Humberside port , said Tom ghoulishly .
I love cats , Tom mused .
' ' Boy , that ' s a bright star , ' ' said Tom seriou

In [6]:
def get_prefixes(word):
    def load_dict(path):
        ret = set()
        with codecs.open(path, "r", "utf8") as r:
            for line in r:
                ret.add(line[:-1])
        return ret
    am = load_dict("/usr/share/dict/american-english")
    br = load_dict("/usr/share/dict/british-english")
    ret = []
    x = word
    for x in iter(lambda: x[:-1], ''):
        if (x in am or x in br) and len(x) > 2:
            ret.append(x)
    return ret

In [7]:
def get_cmu_tree(word, threshold):

    stemmer = PorterStemmer()
    cmu_trie = get_phoenetic_trie()
    if stemmer.stem(word) in cmu:
        new_word = stemmer.stem(word)
    elif word in cmu:
        new_word = word
    else:
        return get_prefixes(word)
    tmp_ret = []
    
    print(new_word)
    for phoneme in cmu[new_word]:
        p = phoneme
        sub_ret = []
        while len(p) > threshold:
            sub_sub_ret = []
            if p in cmu_trie:
                print('Phoneme: ', p)
                sub_sub_ret.extend([i for i in cmu_trie[p] if i != word])
            else:
                sub_sub_ret.extend([])
            sub_sub_ret.extend(list(itertools.chain(*[v for k, v in cmu_trie.iteritems(p, shallow=True)
                                                      if len(k) == len(p)+1 and word not in v])))
            p = p[:-1]
            sub_ret.append(sub_sub_ret)
        tmp_ret.append(sub_ret)
    print(tmp_ret)
    # print(tmp_ret)
    ret = []
    for items in itertools.zip_longest(*tmp_ret):
        ret.append(list(itertools.chain(*items)))

    # ret = list(itertools.chain(*ret[:threshold]))
    ret = list(itertools.chain(*ret))

    ret.extend(get_prefixes(word))
    return list(set(ret))

In [63]:
get_cmu_tree('saucily', 5)

[]

In [9]:
def get_phoenetic_trie():
    dd = defaultdict(list)
    tree = trie.Trie()
    for k, vs in cmu.items():
        for v in vs:
            dd[','.join(v)].append(k)
    for k, v in dd.items():
        tree[k.split(',')] = v
    return tree

In [57]:
stemmer = PorterStemmer()
def prefixes(word, threshold=None):
    
    if not threshold:
        stem = stemmer.stem(word)
        translation = phonetic_translation(stem)
        if stem in cmu:
            threshold = len(translation)
        else:
            threshold = len(translation) - 1
        
    ret = defaultdict(list)
    phonetics = phonetic_translation(word)
    seen = {word}
    

    for i in range(threshold, len(phonetics)):
        for k, v in cmu.items():
            if phonetics[:i] == v[0][:i]:
                
                if k not in seen:
                    ret[i].append(k)
                    seen.add(k)
    return ret

In [62]:
prefixes('whined')

defaultdict(list,
            {3: ['winey',
              'weinberger',
              'weinstein',
              'wine',
              'weiner',
              'whines',
              'windup',
              'weintraub',
              'weinfeld',
              "weinberger's",
              'winer',
              "wine's",
              'whinery',
              'weinroth',
              'weinstock',
              'weinhardt',
              'weindel',
              'weinhold',
              'weinmann',
              'wineland',
              'weinreb',
              'winegarden',
              'wineman',
              'weinman',
              'weinberg',
              'wynona',
              'whiner',
              "weintz'",
              'wines',
              'whining',
              "weinstein's",
              'winery',
              'weinrich',
              'wyne',
              'wineheim',
              'weinreich',
              'wineberg',
              "winery's",
             

In [32]:
'canne' in cmu

False

In [47]:
stemmer.stem('archly')

'archli'

In [51]:
stemmer.stem('phlegm')

'danger'

In [61]:
'phlegm' in cmu

False